In [283]:
# Save the updated script with DOF prints added
# Import coerenti con il tuo stile
from pyomo.environ import ConcreteModel, SolverFactory, value
from idaes.core import FlowsheetBlock, MaterialBalanceType
from idaes.models.unit_models import Translator, Separator, Mixer
from idaes.models.properties.modular_properties.base.generic_property import GenericParameterBlock
from idaes.models_extra.power_generation.properties.natural_gas_PR import get_prop, get_rxn, EosType
import pyomo.environ as pyo
import idaes.core.util.scaling as iscale
import idaes.core.util.model_statistics as stattools
from idaes.core.util.initialization import propagate_state
import idaes.logger as idaeslog
from idaes.core.util.model_statistics import degrees_of_freedom
import idaes.models.unit_models as um
from pyomo.network import Arc
from idaes.models.properties.modular_properties.base.generic_reaction import GenericReactionParameterBlock
from idaes.core.util.model_statistics import (
    degrees_of_freedom, 
    report_statistics,
    activated_constraints_set,
    activated_equalities_set,
    unfixed_variables_set,
    fixed_variables_set,
    variables_set
)

In [284]:
# Logger silenzioso
idaeslog.getLogger("idaes.init").setLevel(idaeslog.ERROR)
idaeslog.getLogger("idaes.solve").setLevel(idaeslog.ERROR)

# Modello
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

# Proprietà
fuel_comps = {"H2": 0.99, "H2O": 0.01}
air_comps = {"O2": 0.21, "N2": 0.79}
reaction_comps = {"H2", "H2O", "O2"}

m.fs.fuel_props = GenericParameterBlock(**get_prop(fuel_comps, phases={"Vap"}, eos=EosType.PR))
m.fs.air_props = GenericParameterBlock(**get_prop(air_comps, phases={"Vap"}, eos=EosType.PR))
m.fs.reaction_props = GenericParameterBlock(**get_prop(reaction_comps, phases={"Vap"}, eos=EosType.PR))

def print_stream(unit, label, stream):
    separator = '=' * 60
    subsection = '-' * 60

    print(f"\n{separator}")
    print(f"{label:^60}")
    print(f"{separator}")

    print("\nSTREAM PROPERTIES:")
    print(f"{'Flow [mol/s]':<20}: {value(stream.flow_mol[0]):.6e}")
    print(f"{'Temperature [K]':<20}: {value(stream.temperature[0]):.2f}")
    print(f"{'Pressure [Pa]':<20}: {value(stream.pressure[0]):.2f}")

    print("\nMOLE FRACTIONS:")
    for t, c in stream.mole_frac_comp.keys():
        print(f"  x_{c:<10}: {value(stream.mole_frac_comp[t, c]):.6e}")

    print(f"\nDOF after {label}: {degrees_of_freedom(unit)}")

    print(f"\n{subsection}")
    print("MODEL STATISTICS REPORT:")
    print(subsection)
    report_statistics(unit)

    print(f"\n{subsection}")
    print("VARIABLES:")
    print(subsection)
    for i, v in enumerate(variables_set(unit)):
        print(f" {i+1} - {v}")

    print(f"\n{subsection}")
    print("ACTIVATED CONSTRAINTS:")
    print(subsection)
    for i, c in enumerate(activated_constraints_set(unit)):
        print(f" {i+1} - {c}")

    print(f"\n{subsection}")
    print("ACTIVATED EQUALITIES:")
    print(subsection)
    for i, eq in enumerate(activated_equalities_set(unit)):
        print(f" {i+1} - {eq}")

    print(f"\n{subsection}")
    print("UNFIXED VARIABLES:")
    print(subsection)
    for i, ufv in enumerate(unfixed_variables_set(unit)):
        print(f" {i+1} - {ufv}")

    print(f"\n{subsection}")
    print("FIXED VARIABLES:")
    print(subsection)
    for i, fv in enumerate(fixed_variables_set(unit)):
        print(f" {i+1} - {fv}")

    print(separator)
    print("\n")


In [285]:
T_fuel_in = 668.9 + 273.15  # = 942.05 K
T_fuel_out = 740.0 + 273.15  # = 1013.15 K
T_tot_in = 675.3 + 273.15  # = 948.45 K
T_air_in = 675.3 + 273.15  # = 948.45 K
T_air_out = 740.0 + 273.15  # = 1013.15 K
T_tot_out = 740.0 + 273.15  # = 1013.15 K

# If you want, you can also store the summed values directly:
T_fuel_in = 942.05
T_fuel_out = 1013.15
T_tot_in = 948.45
T_air_in = 948.45
T_air_out = 1013.15
T_tot_out = 1013.15


In [286]:

"""Delete translator_h2 if it already exists to avoid redefinition warning
if hasattr(m.fs, "translator_h2"):
    m.fs.del_component("translator_h2")"""
#Sara: add this to each unit to avoid warning error when running this cell multiple times OR re run the cells above.
"""Even if you’re sure it’s only defined once in your code, Jupyter notebooks keep the model alive across cells, so:
If you rerun a cell that redefines translator_h2, the warning will come back.
The fix (del_component()) is safe and correct, and helps make your notebook idempotent (safe to re-run cells in any order).
If the warning appears, use del_component() — even if you're sure there's only one definition in your code.
The warning is not about your code structure, it’s about the current state of the Python model object (m.fs) in memory."""

"Even if you’re sure it’s only defined once in your code, Jupyter notebooks keep the model alive across cells, so:\nIf you rerun a cell that redefines translator_h2, the warning will come back.\nThe fix (del_component()) is safe and correct, and helps make your notebook idempotent (safe to re-run cells in any order).\nIf the warning appears, use del_component() — even if you're sure there's only one definition in your code.\nThe warning is not about your code structure, it’s about the current state of the Python model object (m.fs) in memory."

In [ ]:

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator H2
# ----------------------------------------------------------------------------------------------------------------

m.fs.translator_h2 = Translator(
    inlet_property_package=m.fs.fuel_props,
    outlet_property_package=m.fs.reaction_props,
    outlet_state_defined=False,
)
m.fs.translator_h2.inlet.flow_mol[0].fix(0.075397) #Sara: change from 0.00015 kg/s to 0.075397 mol/s
m.fs.translator_h2.inlet.temperature[0].fix(942.05) # Sara: T_fuel_in changed based on excel file
m.fs.translator_h2.inlet.pressure[0].fix(101325)
m.fs.translator_h2.inlet.mole_frac_comp[:, "H2"].fix(0.99)
m.fs.translator_h2.inlet.mole_frac_comp[:, "H2O"].fix(0.01)

@m.fs.translator_h2.Constraint(m.fs.time)
def temp_eqn_h2(b, t): 
    return b.properties_out[t].temperature == b.properties_in[t].temperature

@m.fs.translator_h2.Constraint(m.fs.time)
def press_eqn_h2(b, t): 
    return b.properties_out[t].pressure == b.properties_in[t].pressure

@m.fs.translator_h2.Constraint(m.fs.time)
def flow_eqn_h2(b, t): 
    return b.properties_out[t].flow_mol == b.properties_in[t].flow_mol

@m.fs.translator_h2.Constraint(m.fs.time, list(m.fs.translator_h2.properties_in[0].mole_frac_comp.keys()))
def mole_frac_eqn_h2(b, t, j): 
    return b.properties_out[t].mole_frac_comp[j] == b.properties_in[t].mole_frac_comp[j]


# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Separator Air in o2_rich e o2_poor
# ----------------------------------------------------------------------------------------------------------------
m.fs.separator = Separator(
    property_package=m.fs.air_props,
    outlet_list=["o2_rich_strm", "o2_poor_strm"],
    split_basis=um.SplittingType.componentFlow,
)
m.fs.separator.inlet.flow_mol[0].fix(0.075397 * 10) #Sara: change from 0.00015 kg/s to 0.075397 mol/s
m.fs.separator.inlet.temperature[0].fix(948.45) #Sara: T_air_in = 948.45 K
m.fs.separator.inlet.pressure[0].fix(101325)
m.fs.separator.inlet.mole_frac_comp[0, "O2"].fix(0.21)
m.fs.separator.inlet.mole_frac_comp[0, "N2"].fix(0.79)
m.fs.separator.split_fraction[0, "o2_rich_strm", "O2"].fix(1.0)
m.fs.separator.split_fraction[0, "o2_rich_strm", "N2"].fix(0.0)


# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: translator o2_rich da separator a reaction_props NON FUNZIONAAAAAAAAAAAAAAAAAA
# ----------------------------------------------------------------------------------------------------------------
m.fs.translator_o2 = Translator(
    inlet_property_package=m.fs.air_props,
    outlet_property_package=m.fs.reaction_props,
    outlet_state_defined=False,
)
m.fs.o2_to_translator = Arc(source=m.fs.separator.o2_rich_strm, destination=m.fs.translator_o2.inlet)
pyo.TransformationFactory("network.expand_arcs").apply_to(m)

# ── 1.  Kill N2 in the inlet (must be done *before* solver) ──────────────────
m.fs.translator_o2.inlet.mole_frac_comp[0, "N2"].fix(0.0)
#m.fs.translator_o2.inlet.mole_frac_comp[0, "O2"].fix(1.0)

#Sara: added temp and pressure. DOF from 4 tdown to 2 but does it make sense?
m.fs.translator_o2.inlet.temperature[0].fix(948.45)
m.fs.translator_o2.inlet.pressure[0].fix(101325)

m.fs.translator_o2.properties_out[0].temperature.fix(948.45)
m.fs.translator_o2.properties_out[0].pressure.fix(101325)
m.fs.separator.inlet.flow_mol[0].fix(1.583337e-01)
m.fs.separator.outlet.flow_mol[0].fix(1.583337e-01)
for t in m.fs.time:
    m.fs.translator_o2.properties_out[t].mole_frac_comp["O2"].fix(1.0)
    m.fs.translator_o2.properties_out[t].mole_frac_comp["H2"].fix(1e-19)
    m.fs.translator_o2.properties_out[t].mole_frac_comp["H2O"].fix(1e-19)

#Sara: dall'output sembra che non prenda in considerazione i constraint ftp
#sembra non conoscere le properties in e per questo non usa questi constraint.
@m.fs.translator_o2.Constraint(m.fs.time)
def temp_eqn_o2(b, t): 
    return b.properties_out[t].temperature == b.properties_in[t].temperature

@m.fs.translator_o2.Constraint(m.fs.time)
def press_eqn_o2(b, t): 
    return b.properties_out[t].pressure == b.properties_in[t].pressure

@m.fs.translator_o2.Constraint(m.fs.time)
def flow_eqn_o2(b, t): 
    return b.properties_out[t].flow_mol == b.properties_in[t].flow_mol


# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Mixer per H2 e O2 inlet
# ----------------------------------------------------------------------------------------------------------------
m.fs.mixer = um.Mixer(
    inlet_list=["fuel_strm","o2_rich_strm"],
    momentum_mixing_type=um.MomentumMixingType.none,
    property_package=m.fs.reaction_props,
)
m.fs.mix_h2 = Arc(source=m.fs.translator_h2.outlet, destination=m.fs.mixer.fuel_strm)
m.fs.mix_o2 = Arc(source=m.fs.translator_o2.outlet, destination=m.fs.mixer.o2_rich_strm)

pyo.TransformationFactory("network.expand_arcs").apply_to(m)

@m.fs.mixer.Constraint(m.fs.time)
def pressure_eqn(b, t):
    return b.mixed_state[t].pressure == b.o2_rich_strm_state[t].pressure


# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Stochiometric Reactor
# ----------------------------------------------------------------------------------------------------------------
m.fs.rxn_props = GenericReactionParameterBlock(**get_rxn(m.fs.reaction_props, {"h2_cmb"}))
m.fs.reactor = um.StoichiometricReactor(
    property_package=m.fs.reaction_props,
    reaction_package=m.fs.rxn_props,
    has_pressure_change=False,
    has_heat_transfer=True,
)

m.fs.mix_to_reactor = Arc(source=m.fs.mixer.outlet, destination=m.fs.reactor.inlet)
pyo.TransformationFactory("network.expand_arcs").apply_to(m)


# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Separator water and O2
# ----------------------------------------------------------------------------------------------------------------
m.fs.reactor_separator = Separator(
    property_package=m.fs.reaction_props,
    outlet_list=['water_strm', 'o2_strm'],
    split_basis=um.SplittingType.componentFlow,
)
m.fs.react_to_sep = Arc(source=m.fs.reactor.outlet, destination=m.fs.reactor_separator.inlet)
pyo.TransformationFactory("network.expand_arcs").apply_to(m)

for j in ["H2", "H2O"]:
    m.fs.reactor_separator.split_fraction[0, "water_strm", j].fix(1.0)
m.fs.reactor_separator.split_fraction[0, "o2_strm", "O2"].fix(1.0)


# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator for H2O e H2 stream
# ----------------------------------------------------------------------------------------------------------------
m.fs.translator_h2_out = Translator(
    inlet_property_package=m.fs.reaction_props,
    outlet_property_package=m.fs.fuel_props,
    outlet_state_defined=False,
)

# Connessione dallo stream "water_strm" del separatore
m.fs.water_to_translator = Arc(
    source=m.fs.reactor_separator.water_strm,
    destination=m.fs.translator_h2_out.inlet
)

pyo.TransformationFactory("network.expand_arcs").apply_to(m)

# Constraint FTP standard (totale)
@m.fs.translator_h2_out.Constraint(m.fs.time)
def flow_eqn_h2_out(b, t):
    return b.properties_out[t].flow_mol == b.properties_in[t].flow_mol

@m.fs.translator_h2_out.Constraint(m.fs.time)
def temp_eqn_h2_out(b, t):
    return b.properties_out[t].temperature == b.properties_in[t].temperature

@m.fs.translator_h2_out.Constraint(m.fs.time)
def press_eqn_h2_out(b, t):
    return b.properties_out[t].pressure == b.properties_in[t].pressure

# Componenti comuni
comps_out = set(m.fs.translator_h2_out.properties_out[0].mole_frac_comp.keys())
comps_out.remove("H2")
# SOLO constraint flussi molari individuali per componenti comuni
@m.fs.translator_h2_out.Constraint(m.fs.time, comps_out)
def component_flow_eqn_h2_out(b, t, j):
    return (
                b.properties_out[t].mole_frac_comp[j]
                == b.properties_in[t].mole_frac_comp[j]
            )

#Sara: 
# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator for O2 stream out of the reactor (non reacted)
# ----------------------------------------------------------------------------------------------------------------
m.fs.translator_o2_out = Translator(
    inlet_property_package=m.fs.reaction_props,
    outlet_property_package=m.fs.air_props,
    outlet_state_defined=False,
)

# Connessione dallo stream "o2_rich_strm" del separatore
m.fs.air_to_translator = Arc(                   #called air_to_translator because o2_to_translator is already used
    source=m.fs.reactor_separator.o2_strm,
    destination=m.fs.translator_o2_out.inlet
)

pyo.TransformationFactory("network.expand_arcs").apply_to(m)

# Constraint FTP standard (totale)
@m.fs.translator_o2_out.Constraint(m.fs.time)
def flow_eqn_o2_out(b, t):
    return b.properties_out[t].flow_mol == b.properties_in[t].flow_mol

@m.fs.translator_o2_out.Constraint(m.fs.time)
def temp_eqn_o2_out(b, t):
    return b.properties_out[t].temperature == b.properties_in[t].temperature

@m.fs.translator_o2_out.Constraint(m.fs.time)
def press_eqn_o2_out(b, t):
    return b.properties_out[t].pressure == b.properties_in[t].pressure


comps = set(m.fs.translator_o2_out.properties_out[0].mole_frac_comp.keys())
comps.remove("O2")

@m.fs.translator_o2_out.Constraint(m.fs.time, comps)
def mole_frac_comp_eqn(b, t, c):
    return b.properties_out[t].mole_frac_comp[c] == 1e-19

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Mixer for O2 stream out of the reactor (non reacted) and o2_poor stream from the inlet
# ----------------------------------------------------------------------------------------------------------------

m.fs.mixer2 = um.Mixer(
    inlet_list=["o2_strm","o2_poor_strm"],
    momentum_mixing_type=um.MomentumMixingType.none,
    property_package=m.fs.air_props,
)
m.fs.mix_o2_poor_strm = Arc(source=m.fs.separator.o2_poor_strm, destination=m.fs.mixer2.o2_poor_strm)
m.fs.mix_o2_strm = Arc(source=m.fs.translator_o2_out.outlet, destination=m.fs.mixer2.o2_strm)

pyo.TransformationFactory("network.expand_arcs").apply_to(m)

@m.fs.mixer2.Constraint(m.fs.time)
def pressure_eqn(b, t):
    return b.mixed_state[t].pressure == b.o2_poor_strm_state[t].pressure

WARNING (W1002): Setting Var
'fs.translator_o2.properties_in[0.0].mole_frac_comp[N2]' to a numeric value
`0.0` outside the bounds (1e-20, 1.001).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002


In [282]:


m.fs.translator_h2.initialize()
propagate_state(m.fs.mix_h2)
print_stream(m.fs.translator_h2, "Translator H2 - Inlet", m.fs.translator_h2.inlet)
print_stream(m.fs.translator_h2, "Translator H2 - Outlet", m.fs.translator_h2.outlet)



m.fs.separator.initialize()
propagate_state(m.fs.o2_to_translator)
print_stream(m.fs.separator, "Separator Air - Inlet", m.fs.separator.inlet)
print_stream(m.fs.separator, "Separator Air - O2 Rich", m.fs.separator.o2_rich_strm)
print_stream(m.fs.separator, "Separator Air - O2 Poor", m.fs.separator.o2_poor_strm)



m.fs.translator_o2.initialize()
propagate_state(m.fs.mix_o2)
print_stream(m.fs.translator_o2, "Translator O2 - Inlet", m.fs.translator_o2.inlet)
print_stream(m.fs.translator_o2, "Translator O2 - Outlet", m.fs.translator_o2.outlet)



m.fs.mixer.initialize()
propagate_state(m.fs.mix_to_reactor)
print_stream(m.fs.mixer, "Mixer - O2 Inlet", m.fs.mixer.o2_rich_strm)
print_stream(m.fs.mixer, "Mixer - H2 Inlet", m.fs.mixer.fuel_strm)
print_stream(m.fs.mixer, "Mixer - Mixed Outlet", m.fs.mixer.outlet)




m.fs.reactor.initialize()
propagate_state(m.fs.react_to_sep)
print_stream(m.fs.reactor, "Reactor - Inlet", m.fs.reactor.inlet)
print_stream(m.fs.reactor, "Reactor - Outlet", m.fs.reactor.outlet)




m.fs.reactor_separator.initialize()
propagate_state(m.fs.water_to_translator)
print_stream(m.fs.reactor_separator, "Separator - O2 Stream", m.fs.reactor_separator.o2_strm)
print_stream(m.fs.reactor_separator, "Separator - Water Stream", m.fs.reactor_separator.water_strm)

#Sara:

m.fs.translator_h2_out.initialize()
print_stream(m.fs.translator_h2_out, "Translator H2 out - Inlet", m.fs.translator_h2_out.inlet)
print_stream(m.fs.translator_h2_out, "Translator H2 out - Outlet", m.fs.translator_h2_out.outlet)


m.fs.translator_o2_out.initialize()
propagate_state(m.fs.mix_o2_strm)
print_stream(m.fs.translator_o2_out, "Translator O2 out - Inlet", m.fs.translator_o2_out.inlet)
print_stream(m.fs.translator_o2_out, "Translator O2 out - Inlet", m.fs.translator_o2_out.outlet)

m.fs.mixer2.initialize()
print_stream(m.fs.mixer2, "Mixer - O2 stream Inlet", m.fs.mixer2.o2_strm)
print_stream(m.fs.mixer2, "Mixer - O2 poor stram Inlet", m.fs.mixer2.o2_poor_strm)
print_stream(m.fs.mixer2, "Mixer - Mixed Outlet", m.fs.mixer2.outlet)



                   Translator H2 - Inlet                    

STREAM PROPERTIES:
Flow [mol/s]        : 7.539700e-02
Temperature [K]     : 942.05
Pressure [Pa]       : 101325.00

MOLE FRACTIONS:
  x_H2        : 9.900000e-01
  x_H2O       : 1.000000e-02

DOF after Translator H2 - Inlet: 0

------------------------------------------------------------
MODEL STATISTICS REPORT:
------------------------------------------------------------

Model Statistics  -  fs.translator_h2 

Degrees of Freedom: 0 

Total No. Variables: 20 
    No. Fixed Variables: 5
    No. Unused Variables: 0 (Fixed):0)
    No. Variables only in Inequalities: 0 (Fixed: 0) 

Total No. Constraints: 15 
    No. Equality Constraints: 15 (Deactivated: 0)
    No. Inequality Constraints: 0 (Deactivated: 0)

No. Objectives: 0 (Deactivated: 0)

No. Blocks: 3 (Deactivated: 0) 
No. Expressions: 65 


------------------------------------------------------------
VARIABLES:
------------------------------------------------------------